In [7]:
import os

cwd = os.path.abspath("")
cwd

'/Users/jskaf/BBLLM-Goodfire-Hackathon'

In [9]:
THRESHOLD_EDGE = 3
PATH_GRAPH = os.path.join(cwd, "data", "graph.weighted")
PATH_GRAPH

'/Users/jskaf/BBLLM-Goodfire-Hackathon/data/graph.weighted'

In [23]:
import networkx as nx

G = nx.read_weighted_edgelist(PATH_GRAPH, nodetype=int)

In [24]:
print(len(G.nodes), len(G.edges))

32642 612345


In [25]:
nx.density(G)

0.001149439072249825

In [26]:
G.remove_edges_from([(u, v) for u, v, w in G.edges(data=True) if w['weight'] < THRESHOLD_EDGE])

In [27]:
print(len(G.nodes), len(G.edges))

32642 81188


In [28]:
nx.density(G)

0.00015239882647497536

In [29]:
nb_composantes = nx.number_connected_components(G)
nb_noeuds_total = G.number_of_nodes()
print(f'Nombre de composantes connectées : {nb_composantes}')
print(f'Nombre de composantes déconnectées : {nb_noeuds_total - nb_composantes}')
largest_cc = max(nx.connected_components(G), key=len)
sous_graphe = G.subgraph(largest_cc).copy() 

Nombre de composantes connectées : 10252
Nombre de composantes déconnectées : 22390


In [32]:
print(len(sous_graphe.nodes), len(sous_graphe.edges))

21983 80712


In [33]:
nx.density(sous_graphe)

0.00033405189615735734

In [34]:
MAPPING_FILE = os.path.join(cwd, "data", "index_label_mapping.json")
import json

with open(MAPPING_FILE, 'r') as f:
    node_mapping = json.load(f)

mapping = {int(k): v for k, v in node_mapping.items()}
    
final_graph = nx.relabel_nodes(sous_graphe, mapping)

In [35]:
final_graph.nodes(data=True)

NodeDataView({'Beginning of a new statement or scenario with an indefinite article': {}, "Sentence/description starter token, especially capitalized 'A'": {}, 'Question or problem statement introduction in educational contexts': {}, 'Beginning of task or problem description': {}, 'First-person singular pronoun at sentence start or in self-reference': {}, "First-person pronoun 'I' at the start of requests or statements": {}, 'Quotation marks introducing key phrases or topics': {}, 'Start of quoted text or input for processing': {}, 'Indefinite article detection, especially at the start of clauses': {}, "Sentence-initial definite article 'The'": {}, "Definite article 'the' in various contexts": {}, 'First item or ordinal number one across languages': {}, 'Multilingual summary or conclusion request': {}, 'Multilingual tokens related to marking, branding, or trademarks': {}, 'Female names across languages': {}, 'Common youthful English first names': {}, 'Introduction of a series of questio

In [36]:
import ipysigma

ipysigma.Sigma(final_graph,   node_metrics={"community": {"name": "louvain", "resolution": 1.5}},
  node_color="community", node_size=sous_graphe.degree, edge_size = 'weight')

Sigma(nx.Graph with 21,107 nodes and 79,689 edges)

In [38]:
#Save the graph
nx.write_weighted_edgelist(sous_graphe, os.path.join(cwd, "data", "graph_filtered.weighted"))